In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from PIL import Image
# import os

# # Define the path to the directory containing your images
# input_dir = 'Resized'

# # Define the path to the directory where augmented images will be saved
# output_dir = 'BackgroundRemoved'

# # Create the output directory if it doesn't exist
# os.makedirs(output_dir, exist_ok=True)

# # Initialize the ImageDataGenerator with desired augmentations
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# # List all image files in the input directory
# image_files = [os.path.join(input_dir, filename) for filename in os.listdir(input_dir) if filename.endswith(('.jpg', '.jpeg', '.png'))]

# # Perform augmentation for each image
# for image_file in image_files:
#     img = Image.open(image_file)
#     img_array = img.convert('RGB')

#     # Reshape the image to (1, height, width, channels) for the generator
#     img_array = img_array.resize((256, 256))  # adjust the size as needed
#     img_array = img_array.reshape((1,) + img_array.shape)

#     # Generate augmented images
#     i = 0
#     for batch in datagen.flow(img_array, batch_size=1, save_to_dir=output_dir, save_prefix='aug', save_format='jpeg'):
#         i += 1
#         if i >= 5:  # You can adjust the number of augmented images per original image
#             break


In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.preprocessing import image
# import os

# # Set your base directory containing the subdirectories with images
# base_directory = 'Resized'

# # Specify the augmentation parameters
# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     shear_range=0.1,
#     zoom_range=0.1,
#     horizontal_flip=True,
#     fill_mode='nearest'
# )

# # Specify the directory to save the augmented images
# output_directory = 'Augmented1'

# # Specify the number of subdirectories to augment
# num_subdirectories_to_augment = 3

# # Counter to keep track of the number of processed subdirectories
# subdirectory_count = 0

# # Loop through each subdirectory in the base directory
# for subdirectory in os.listdir(base_directory):
#     subdirectory_path = os.path.join(base_directory, subdirectory)

#     # Check if the item is a directory
#     if os.path.isdir(subdirectory_path):
#         print(f"Augmenting images in {subdirectory}...")

#         # Create a new directory in the output directory for the augmented images
#         output_subdirectory = os.path.join(output_directory, subdirectory + '_augmented')
#         os.makedirs(output_subdirectory, exist_ok=True)

#         # Generate augmented images and save them to the new directory
#         for filename in os.listdir(subdirectory_path):
#             img_path = os.path.join(subdirectory_path, filename)

#             # Check if the item is a file (image)
#             if os.path.isfile(img_path):
#                 img = image.load_img(img_path, target_size=(256, 256))
#                 x = image.img_to_array(img)
#                 x = x.reshape((1,) + x.shape)

#                 # Generate augmented images and save them to the new directory
#                 i = 0
#                 for batch in datagen.flow(x, batch_size=1, save_to_dir=output_subdirectory, save_prefix='aug', save_format='jpeg'):
#                     i += 1
#                     if i >= 2:  # Generate 5 augmented images per original image
#                         break

#         subdirectory_count += 1

#         # Check if the required number of subdirectories have been augmented
#         if subdirectory_count >= num_subdirectories_to_augment:
#             break

# print(f"Data augmentation completed for the first {num_subdirectories_to_augment} subdirectories. Augmented images saved to separate directories.")

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models

# Define directories for train and validation datasets
train_dir = '/path/to/train/directory'
test_dir = '/path/to/test/directory'

# Define data generators for train and validation datasets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Set the image size and batch size
img_size = (256, 256)
batch_size = 32

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load validation data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the InceptionV3 model pretrained on ImageNet
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = layers.Flatten()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(3, activation='softmax')(x)

# Combine the base model with custom classification head
model = models.Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Found 536 images belonging to 3 classes.
Found 203 images belonging to 3 classes.
87924736/87910968 [==============================] - 16s 0us/step
Epoch 1/10
17/17 [==============================] - 124s 6s/step - loss: 16.8296 - accuracy: 0.5243 - val_loss: 1.8383 - val_accuracy: 0.8621
Epoch 2/10
17/17 [==============================] - 98s 6s/step - loss: 3.6019 - accuracy: 0.7425 - val_loss: 0.4197 - val_accuracy: 0.9113
Epoch 3/10
17/17 [==============================] - 99s 6s/step - loss: 0.7512 - accuracy: 0.8265 - val_loss: 0.1559 - val_accuracy: 0.9458
Epoch 4/10
17/17 [==============================] - 127s 7s/step - loss: 0.4712 - accuracy: 0.8097 - val_loss: 0.2033 - val_accuracy: 0.9261
Epoch 5/10
17/17 [==============================] - 102s 6s/step - loss: 0.3654 - accuracy: 0.8582 - val_loss: 0.1848 - val_accuracy: 0.9409
Epoch 6/10
17/17 [==============================] - 109s 6s/step - loss: 0.3426 - accuracy: 0.8806 - val_loss: 0.2043 - val_accuracy: 0.9360
Epoch 7